<a href="https://colab.research.google.com/github/VesalAhsani/Driver-behavior-detection/blob/main/DMS_YOLOv8s_Focal_224_RGB%26GRAY_mergedclasses_TruePretrained_062025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv8-s Classification @ 224px on Colab
No Augmentation!

REAL Focal with alpha cap and higher gamma!!

Pretrained = True..

Some of the classes are merged here for training rather than mapping in inference !

In [ ]:
# @title 1️⃣ Install dependencies
# Colab comes with torch+cuda preinstalled, but we upgrade ultralytics.
!pip install --upgrade ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.7 MB/s eta 0:00:00


In [ ]:
# @title 2️⃣ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title 3️⃣ Paths & Unzip Dataset
import zipfile, os, glob
from pathlib import Path

# ── CONFIG ──────────────────────────────────────
ZIP_PATH    = "/content/drive/MyDrive/Split224_rgbgray_mergedclasses.zip"
EXTRACT_DIR = Path("/content/dataset224")
DATA_DIR    = EXTRACT_DIR / "Split224_rgbgray_mergedclasses"   # <-- must contain train/ & val/
# ────────────────────────────────────────────────

# 3.1 Unzip if needed
if not DATA_DIR.exists():
    print("📦 Unzipping dataset…")
    EXTRACT_DIR.mkdir(exist_ok=True)
    with zipfile.ZipFile(ZIP_PATH, 'r') as zf:
        zf.extractall(EXTRACT_DIR)
    print("✅ Unzipped to", EXTRACT_DIR)

# 3.2 Print counts per class
print("\n🧾 Dataset composition:")
for split in ("train", "val"):
    print(f"\n– {split.upper()}:")
    for cls in sorted((DATA_DIR/split).iterdir()):
        if cls.is_dir():
            imgs = list(cls.glob("*.*"))
            print(f"  {cls.name:<20s} {len(imgs):5d}")

📦 Unzipping dataset…
✅ Unzipped to /content/dataset224

🧾 Dataset composition:

– TRAIN:
  Control_Panel         8341
  Eating:Drinking       9937
  Look_Down             3761
  Look_Side            11988
  Makeup                6219
  Normal                8167
  Phone_Usage          24820
  Reaching_Behind       4634
  Sleep                 4039
  Smoking              12355
  Yawning               3206

– VAL:
  Control_Panel          927
  Eating:Drinking       1105
  Look_Down              418
  Look_Side             1334
  Makeup                 691
  Normal                 908
  Phone_Usage           2760
  Reaching_Behind        515
  Sleep                  449
  Smoking               1375
  Yawning                357


In [ ]:
# @title 4️⃣ Compute α (class imbalance) for focal loss

import torch

# count images per class
train_root = DATA_DIR / "train"
class_counts = {
    cls.name: len(list((train_root/cls.name).glob("*.*")))
    for cls in sorted(train_root.iterdir()) if cls.is_dir()
}

# compute inverse-frequency weights, cap at 2.0
max_n     = max(class_counts.values())
raw_alpha = torch.tensor([max_n / class_counts[c] for c in class_counts],
                         dtype=torch.float32)
alpha     = torch.minimum(raw_alpha, torch.tensor(2.0)).cuda()

print("\n🎯 α weights (per class, clipped):")
for name, w in zip(class_counts.keys(), alpha):
    print(f"  {name:<20s} {w:.3f}")


🎯 α weights (per class, clipped):
  Control_Panel        2.000
  Eating:Drinking      2.000
  Look_Down            2.000
  Look_Side            2.000
  Makeup               2.000
  Normal               2.000
  Phone_Usage          1.000
  Reaching_Behind      2.000
  Sleep                2.000
  Smoking              2.000
  Yawning              2.000


In [ ]:
# @title 5️⃣ Define FocalLoss & Train YOLOv8-s-cls

import torch
import torch.nn as nn
import shutil
from pathlib import Path
from ultralytics import YOLO

# ─── Focal Loss ───────────────────────────────
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super().__init__()
        self.gamma, self.alpha, self.reduction = gamma, alpha, reduction

    def forward(self, inputs, targets):
        logp = torch.nn.functional.log_softmax(inputs, dim=-1)
        p    = torch.exp(logp)
        t    = targets.long()
        pt   = p.gather(-1, t.unsqueeze(-1)).squeeze(-1)
        focal = (1 - pt) ** self.gamma
        loss = -focal * logp.gather(-1, t.unsqueeze(-1)).squeeze(-1)
        if self.alpha is not None:
            loss = loss * self.alpha[t]
        return loss.mean() if self.reduction=='mean' else loss.sum()



# ── paths for results ---------------------------------------------------
run_name    = "yolov8s_focal224"
RESULTS_DIR = "/content/drive/MyDrive/yolov8s_cls_224_mergedclasses_results"

# ─── Load model & override loss ─────────────────
print("🚀 Loading YOLOv8s-cls pretrained weights…")
model = YOLO("yolov8s-cls.pt")
alpha = alpha.to(model.device)
model.model.loss_fn = FocalLoss(gamma=2.0, alpha=alpha)
print("🔍 Active loss_fn:", model.model.loss_fn)

# ─── Train ────────────────────────────────────
print("\n📚 Starting training…")
model.train(
    data=str(DATA_DIR),     # expects train/ & val/ inside
    epochs=75,
    batch=32,
    imgsz=224,
    lr0=5e-4,
    optimizer="Adam",
    project=RESULTS_DIR,
    name=run_name,
    pretrained=True,
    fliplr=0.5,
    flipud=0,
)


# ─── Validate & copy best.pt ──────────────────
print("\n📊 Running final validation…")
metrics = model.val()

src = os.path.join(model.trainer.save_dir, "weights", "best.pt")
dst = os.path.join(RESULTS_DIR, run_name, "weights", "best.pt")
os.makedirs(os.path.dirname(dst), exist_ok=True)
os.system(f"cp {src} {dst}")

print("✅ Best weights saved to:", dst)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🚀 Loading YOLOv8s-cls pretrained weights…


100%|██████████| 12.3M/12.3M [00:00<00:00, 493MB/s]


🔍 Active loss_fn: FocalLoss()

📚 Starting training…
Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset224/Split224_rgbgray_mergedclasses, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=75, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_focal224, nbs=64, nms=False, opset=None, optimize=False, optimizer

100%|██████████| 5.35M/5.35M [00:00<00:00, 446MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 539.9±244.3 MB/s, size: 14.1 KB)


...


75 epochs completed in 2.469 hours.
Optimizer stripped from /content/drive/MyDrive/yolov8s_cls_224_mergedclasses_results/yolov8s_focal224/weights/last.pt, 10.3MB
Optimizer stripped from /content/drive/MyDrive/yolov8s_cls_224_mergedclasses_results/yolov8s_focal224/weights/best.pt, 10.3MB

Validating /content/drive/MyDrive/yolov8s_cls_224_mergedclasses_results/yolov8s_focal224/weights/best.pt...
Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLOv8s-cls summary (fused): 30 layers, 5,089,291 parameters, 0 gradients, 12.5 GFLOPs
train: /content/dataset224/Split224_rgbgray_mergedclasses/train... found 97467 images in 11 classes ✅ 
val: /content/dataset224/Split224_rgbgray_mergedclasses/val... found 10839 images in 11 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 170/170 [00:03<00:00, 45.17it/s]


                   all      0.994          1
Speed: 0.1ms preprocess, 0.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/drive/MyDrive/yolov8s_cls_224_mergedclasses_results/yolov8s_focal224

📊 Running final validation…
Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLOv8s-cls summary (fused): 30 layers, 5,089,291 parameters, 0 gradients, 12.5 GFLOPs
train: /content/dataset224/Split224_rgbgray_mergedclasses/train... found 97467 images in 11 classes ✅ 
val: /content/dataset224/Split224_rgbgray_mergedclasses/val... found 10839 images in 11 classes ✅ 
test: None...
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 538.5±234.1 MB/s, size: 14.1 KB)


val: Scanning /content/dataset224/Split224_rgbgray_mergedclasses/val... 10839 images, 0 corrupt: 100%|██████████| 10839/10839 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 339/339 [00:04<00:00, 75.62it/s]


                   all      0.994          1
Speed: 0.1ms preprocess, 0.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/drive/MyDrive/yolov8s_cls_224_mergedclasses_results/yolov8s_focal2242
✅ Best weights saved to: /content/drive/MyDrive/yolov8s_cls_224_mergedclasses_results/yolov8s_focal224/weights/best.pt


In [ ]:
import os
import time
from google.colab import runtime

# Wait for training to complete before disconnecting
print("Training completed! Releasing GPU resources...")
time.sleep(20)  # Give some time for final processing

# Automatically disconnect the Colab session
runtime.unassign()
os._exit(0)

Training completed! Releasing GPU resources...
